In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("speech.txt")

document = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000 , chunk_overlap = 30)

docs = text_splitter.split_documents(document)


In [4]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

In [7]:
embeddings = OllamaEmbeddings(
    model='mxbai-embed-large'
)
db = FAISS.from_documents(docs , embeddings)
db

In [9]:
## queryig 
query = "how does the speaker describe the desired outcome of the war?"

docs = db.similarity_search(query)
docs[0].page_content


'It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'

## As a Retreer

In [15]:
retriever = db.as_retriever()
docs =retriever.invoke(query)
docs[0]

Document(id='d8358091-fd8a-4406-a898-a10c8948a275', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.')

### Similarity Search with score 

In [16]:
docs_and_score=db.similarity_search_with_score(query)
docs_and_score  

[(Document(id='d8358091-fd8a-4406-a898-a10c8948a275', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
  np.float32(204.14276)),
 (Document(id='58456883-03bd-4e9e-b492-56af763a3c25', metadata={'source': 'spe

In [17]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[0.5108727216720581,
 -0.3926515281200409,
 -0.28115418553352356,
 0.4480104148387909,
 -0.7402589321136475,
 -1.512639045715332,
 -0.7012879252433777,
 0.33826005458831787,
 0.046920426189899445,
 -0.1355317234992981,
 0.28588199615478516,
 0.05269922316074371,
 -0.3366398811340332,
 0.5878270268440247,
 -0.39368313550949097,
 -0.413349986076355,
 -0.7217355966567993,
 -0.2095656841993332,
 0.2557813227176666,
 0.04954231530427933,
 -0.03380734473466873,
 -0.4084640145301819,
 -0.6066622734069824,
 0.38278234004974365,
 -0.11302552372217178,
 0.273830771446228,
 -0.5984296202659607,
 0.20383551716804504,
 1.1324183940887451,
 -0.11233207583427429,
 -0.1722392737865448,
 0.5426315665245056,
 -0.040869176387786865,
 -0.685344398021698,
 0.43446090817451477,
 -0.63125079870224,
 0.427646279335022,
 -1.9121836423873901,
 0.1352539360523224,
 -0.15927207469940186,
 0.2530469298362732,
 0.28905290365219116,
 0.5580822229385376,
 -0.45594942569732666,
 -1.4730027914047241,
 -0.28254377841949

In [19]:
get_score = db.similarity_search_by_vector(embedding_vector)
get_score

[Document(id='d8358091-fd8a-4406-a898-a10c8948a275', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='58456883-03bd-4e9e-b492-56af763a3c25', metadata={'source': 'speech.txt'}, page_content='…\n

In [20]:
#### Saing and loading 

db.save_local("faiss_index")

In [ ]:
## loading 
new_df = FAISS.load_local("faiss_index",embeddings , allow_dangerous_deserialization=True)

docs = new_df.similarity_search(query)

In [23]:
docs

[Document(id='d8358091-fd8a-4406-a898-a10c8948a275', metadata={'source': 'speech.txt'}, page_content='It is a distressing and oppressive duty, gentlemen of the Congress, which I have performed in thus addressing you. There are, it may be, many months of fiery trial and sacrifice ahead of us. It is a fearful thing to lead this great peaceful people into war, into the most terrible and disastrous of all wars, civilization itself seeming to be in the balance. But the right is more precious than peace, and we shall fight for the things which we have always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='58456883-03bd-4e9e-b492-56af763a3c25', metadata={'source': 'speech.txt'}, page_content='…\n